# Bosonic Double-well

In [ ]:
using KadanoffBaym
using FFTW, Interpolations
using LinearAlgebra

using PyPlot
using PyCall
qt = pyimport("qutip")
np = pyimport("numpy")
sp = pyimport("scipy")

font_size = 16
# font_size = 20

PyPlot.matplotlib.rc("text", usetex=true)
PyPlot.matplotlib.rc("font", family="serif", size=font_size)
PyPlot.matplotlib.rc("axes", labelsize=font_size)
PyPlot.matplotlib.rc("xtick.major", size=8)
PyPlot.matplotlib.rc("ytick.major", size=8)
PyPlot.matplotlib.rc("xtick.minor", visible=true, size=4)
PyPlot.matplotlib.rc("ytick.minor", visible=true, size=4)
PyPlot.matplotlib.rc("xtick", top=true, direction="inout")
PyPlot.matplotlib.rc("ytick", right=true, direction="inout")

In [ ]:
# include("../src_T/SlaveMolecule.jl")

## Model

### Non-Hermitian Hamiltonian:

$$
    \hat{H} = \left(\omega_1 -i\frac{\lambda}{2}\right) a^{\dagger}_1 a^\phantom{\dagger}_1 -i\frac{\gamma}{2} a^\phantom{\dagger}_1 a^{\dagger}_1  + \omega_2 a^{\dagger}_2 a^\phantom{\dagger}_2 + J \left(a^{\dagger}_1 a^\phantom{\dagger}_2 + a^{\dagger}_2 a^\phantom{\dagger}_1\right)
$$

### Master equation:

$$
    \partial_{t} \hat{\rho}=-i\left[\hat{H} \hat{\rho}-\hat{\rho} \hat{H}^{
    \dagger}\right]+\lambda a^\phantom{\dagger}_1 \hat{\rho}  a^{\dagger}_1 +\gamma  a^{\dagger}_1 \hat{\rho} a^\phantom{\dagger}_1
$$

### Equations of motion

#### Vertical Time.

\begin{align}\begin{split}
    0 &= \begin{pmatrix}
    i \partial_t - \omega_1 + i(\lambda + \gamma)/2 & -J \\
    -J & i \partial_t - \omega_2
    \end{pmatrix} 
    \begin{pmatrix}
    G^<_{11} & G^<_{12} \\
    G^<_{21} & G^<_{22}
    \end{pmatrix}(t, t') 
    - i\gamma
    \begin{pmatrix}
    G^{\tilde{T}}_{11} & G^{\tilde{T}}_{12} \\
    0 & 0
    \end{pmatrix}(t, t')  \\
    0 &= \begin{pmatrix}
    i \partial_t - \omega_1 - i(\lambda + \gamma)/2 & -J \\
    -J & i \partial_t - \omega_2
    \end{pmatrix} 
    \begin{pmatrix}
    G^>_{11} & G^>_{12} \\
    G^>_{21} & G^>_{22}
    \end{pmatrix}(t, t')  
    + i\lambda
    \begin{pmatrix}
    G^T_{11} & G^T_{12} \\
    0 & 0
    \end{pmatrix}(t, t')   
\end{split}\end{align}

#### Horizontal Time.

\begin{align}\begin{split}
    0 &= \begin{pmatrix}
    G^<_{11} & G^<_{12} \\
    G^<_{21} & G^<_{22}
    \end{pmatrix}(t, t') 
    \begin{pmatrix}
    i \partial_{t'} + \omega_1 + i(\lambda + \gamma)/2 & J \\
    J & i \partial_{t'} + \omega_2
    \end{pmatrix} 
    - i\gamma
    \begin{pmatrix}
    G^{{T}}_{11} & 0 \\
    G^{{T}}_{21} & 0
    \end{pmatrix}(t, t')     \\
    0 &= \begin{pmatrix}
    G^>_{11} & G^>_{12} \\
    G^>_{21} & G^>_{22}
    \end{pmatrix}(t, t') 
    \begin{pmatrix}
    i \partial_{t'} + \omega_1 - i(\lambda + \gamma)/2 & J \\
    J & i \partial_{t'} + \omega_2
    \end{pmatrix}  
    + i\lambda
    \begin{pmatrix}
    G^{\tilde{T}}_{11} & 0 \\
    G^{\tilde{T}}_{21} & 0
    \end{pmatrix}(t, t')   
\end{split}\end{align}

#### Equal-Time.

\begin{align}\begin{split}
    0 &= \begin{pmatrix}
    i \partial_T + i(\lambda + \gamma) & 0 \\
    0 & i \partial_T
    \end{pmatrix} 
    \begin{pmatrix}
    G^<_{11} & G^<_{12} \\
    G^<_{21} & G^<_{22}
    \end{pmatrix}(T, 0)
    - \left[\begin{pmatrix}
    \omega_1 & J \\
    J & \omega_2
    \end{pmatrix}, 
    \begin{pmatrix}
    G^<_{11} & G^<_{12} \\
    G^<_{21} & G^<_{22}
    \end{pmatrix}(T, 0)\right] 
    - i\gamma
    \begin{pmatrix}
    G^T_{11} + G^{\tilde{T}}_{11} & G^\tilde{T}_{12} \\
    G^{{T}}_{21} & 0
    \end{pmatrix}(T, 0)       \\
    0 &= \begin{pmatrix}
    i \partial_T - i(\lambda + \gamma) & 0 \\
    0 & i \partial_T
    \end{pmatrix} 
    \begin{pmatrix}
    G^>_{11} & G^>_{12} \\
    G^>_{21} & G^>_{22}
    \end{pmatrix}(T, 0)  
    - \left[\begin{pmatrix}
    \omega_1 & J \\
    J & \omega_2
    \end{pmatrix}, 
    \begin{pmatrix}
    G^>_{11} & G^>_{12} \\
    G^>_{21} & G^>_{22}
    \end{pmatrix}(T, 0)\right]
    + i\lambda
    \begin{pmatrix}
    G^T_{11} + G^{\tilde{T}}_{11} & G^T_{12} \\
    G^{\tilde{T}}_{21} & 0
    \end{pmatrix}(T, 0)   
\end{split}\end{align}

## Solving

In [ ]:
# time parameters
T = 16.0

# Non-Hermitian Hamiltonian and jump operator
omega_1 = 5.0
omega_2 = 0.0
J = 0.5*2*pi*1.0/2 
# U = 0*1.0
lambda = 2.0
gamma = lambda

N_1 = 1.
N_2 = 0.1

H = ComplexF64[omega_1 - 0.5im * ((N_1 + 1) * lambda + N_1 * gamma) J; J omega_2 - 0.5im * ((N_2 + 1) * lambda + N_2 * gamma)]
h = H;

In [ ]:
# quantum numbers
L = 2

# Define your Green functions at (t0, t0), time-arguments at the end!
GL = GreenFunction(zeros(ComplexF64, L, L, 1, 1), SkewHermitian)
GG = GreenFunction(zeros(ComplexF64, L, L, 1, 1), SkewHermitian)

# initial condition
N_0 = 2

GL[2, 2, 1, 1] = -1.0im * N_0
GG[2, 2, 1, 1] = -1.0im * (N_0 + 1)
GG[1, 1, 1, 1] = -1.0im

u0 = [GL, GG];

In [ ]:
function θ(t)
   0.5 * (sign(t) + 1)
end

In [ ]:
GT = (t, t′, reg) -> θ(t - t′ + reg) * GG[t, t′] + θ(t′ - t - reg) * GL[t, t′]
GA = (t, t′, reg) -> θ(t - t′ + reg) * GL[t, t′] + θ(t′ - t - reg) * GG[t, t′];

In [ ]:
GT(1, 1, 0) + GA(1, 1, 0) == GG[1, 1] + GL[1, 1]

In [ ]:
GT(1, 1, 1e-15) == GG[1, 1]

In [ ]:
GA(1, 1, 1e-15) == GL[1, 1]

In [ ]:
function ff_vert(u, times, t, t′, reg=1e-15)
    GL, GG = u[1], u[2]
    f_L = -1.0im * (H * GL[t, t′] + [[1.0im * N_1 * gamma, 0] [0, 1.0im * N_2 * gamma]] * GA(t, t′, reg))
    f_G = -1.0im * (adjoint(H) * GG[t, t′] - 1.0im * lambda * [[(N_1 + 1), 0] [0, (N_2 + 1)]] * GT(t, t′, reg))
  return [f_L, f_G]
end

In [ ]:
f_vert(u0, [], 1, 1)

In [ ]:
ff_vert(u0, [], 1, 1)

In [ ]:
ff_vert(u0, [], 1, 1, 0) - adjoint.(ff_vert(u0, [], 1, 1, 0))

In [ ]:
f_diag(u0, [1], 1)

In [ ]:
function f_vert(u, times, t, t′)
    GL, GG = u[1], u[2]
    f_L = -1.0im * (H * GL[t, t′] + [[1.0im * N_1 * gamma, 0] [0, 1.0im * N_2 * gamma]] * GL[t, t′])
    f_G = -1.0im * (adjoint(H) * GG[t, t′] - 1.0im * lambda * [[(N_1 + 1), 0] [0, (N_2 + 1)]] * GG[t, t′])
  return [f_L, f_G]
end

function f_diag(u, times, t)
    println("t: $(times[t])")
    GL, GG = u[1], u[2]
    f_L = -1.0im * (H * GL[t, t] - GL[t, t] * adjoint(H)
        + 1.0im * gamma  * [[N_1 * (GL[1, 1, t, t] + GG[1, 1, t, t]), (N_1 + N_2) * (GL[2, 1, t, t] + GG[2, 1, t, t]) / 2] [(N_1 + N_2) * (GL[1, 2, t, t] + GG[1, 2, t, t]) / 2, N_2 * (GL[2, 2, t, t] + GG[2, 2, t, t])]])
    f_G = -1.0im * (adjoint(H) * GG[t, t] - GG[t, t] * H
        - 1.0im * lambda * [[(N_1 + 1) * (GL[1, 1, t, t] + GG[1, 1, t, t]), (N_1 + N_2 + 2) * (GG[2, 1, t, t] + GL[2, 1, t, t]) / 2] [(N_1 + N_2 + 2) * (GG[1, 2, t, t] + GL[1, 2, t, t]) / 2, (N_2 + 1) * (GL[2, 2, t, t] + GG[2, 2, t, t])]])
  return [f_L, f_G]
end

In [ ]:
# Basically this will mutate Lesser and Greater in place
sol = kbsolve(f_vert, f_diag, u0, (0.0, T); dtini=1e-14, atol=1e-6, rtol=1e-4);#, dtmax=2^-6);

In [ ]:
n = length(sol.t) - 1

In [ ]:
times = sol.t
horizontal_times = k -> vcat(times[k:end] .- times[k], times[end] .+ (1:(k - 1) |> collect) .* (times[end] - times[end-1]));

### Tim

In [ ]:
# n = 2^9
# initial_GL = -1.0im * [0 0; 0 N_0];

# times = range(0, length=n+1, stop=T) |> collect;

# GL_T, GG_T = SlaveMolecule.simulation(n, T/n, lambda, gamma, 0., h, initial_GL);

## QuTiP benchmark

In [ ]:
# maximal number of photons
# Fock-space dimension will be n_max + 1
n_max = 15

# initial state
psi0_list = [qt.basis(n_max + 1, 0), qt.basis(n_max + 1, N_0)]
psi0 = qt.tensor(psi0_list)

# define annihilation operators
a_1_list = [qt.destroy(n_max + 1), qt.qeye(n_max + 1)]
a_1  = qt.tensor(a_1_list)    
a_2_list = [qt.qeye(n_max + 1), qt.destroy(n_max + 1)]
a_2  = qt.tensor(a_2_list)    

# Hamiltonian
H  = J * a_1.dag() * a_2
H += H.dag()
H += omega_1 * a_1.dag() * a_1 + omega_2 * a_2.dag() * a_2
# H += 0.5 * U *(a_1.dag() * a_1 * a_1.dag() * a_1 + a_2.dag() * a_2 * a_2.dag() * a_2)

observables = [a_1.dag()*a_1, a_2.dag()*a_2, a_1*a_1.dag(), a_2*a_2.dag()];

In [ ]:
j_ops = [sqrt((N_1 + 1) * lambda) * a_1, sqrt((N_2 + 1) * lambda) * a_2, 
         sqrt(N_1 * gamma) * a_1.dag(), sqrt(N_2 * gamma) * a_2.dag()] # qutip only needs the jump operator

# quickly solve once for observables
me = qt.mesolve(H, psi0, times, j_ops, observables);

#### Two times

In [ ]:
# # solve for the time-dependent density matrix
# t_sols = qt.mesolve(H, psi0, times, j_ops); # t_sols.states returns density matrices

Calculate $\langle a^\dagger_1(t') a^\phantom{\dagger}_1(t)\rangle$, where $t' = t + \tau$.

In [ ]:
# tau_t_sols = Dict()
# for k in 1:length(t_sols.states)
#     tau_t_sols[k] = qt.mesolve(H, a_1 * t_sols.states[k], horizontal_times(k), j_ops).states
# end

# a_1_dag_a_1 = zeros(ComplexF64, length(t_sols.states), length(t_sols.states))
# for k in 1:length(t_sols.states)
#     for l in 1:length(t_sols.states)
#         a_1_dag_a_1[k, l] = (a_1.dag() * tau_t_sols[k][l]).tr()    
#     end
# end

In [ ]:
# tau_t_sols = Dict()
# for k in 1:length(t_sols.states)
#     tau_t_sols[k] = qt.mesolve(H, a_1.dag() * t_sols.states[k], horizontal_times(k), j_ops).states
# end

# a_1_a_1_dag = zeros(ComplexF64, length(t_sols.states), length(t_sols.states))
# for k in 1:length(t_sols.states)
#     for l in 1:length(t_sols.states)
#         a_1_a_1_dag[k, l] = (a_1 * tau_t_sols[k][l]).tr()    
#     end
# end

In [ ]:
# # reshape the above array to fit into our two-time "matrix" structure 
# # see the plot below for illustration
# unskewed_a_1_dag_a_1 = zeros(ComplexF64, length(t_sols.states), 2*length(t_sols.states) - 1)
# for (k, x) in enumerate([a_1_dag_a_1[k, :] for k in 1:length(t_sols.states)])
#     for (l, y) in enumerate(x)
#         ind = k + l - 1 # verify the -1 relative to the original python code
#         unskewed_a_1_dag_a_1[k, ind] = y  
#     end
# end

In [ ]:
# # reshape the above array to fit into our two-time "matrix" structure 
# # see the plot below for illustration
# unskewed_a_1_a_1_dag = zeros(ComplexF64, length(t_sols.states), 2*length(t_sols.states) - 1)
# for (k, x) in enumerate([a_1_a_1_dag[k, :] for k in 1:length(t_sols.states)])
#     for (l, y) in enumerate(x)
#         ind = k + l - 1 # verify the -1 relative to the original python code
#         unskewed_a_1_a_1_dag[k, ind] = y  
#     end
# end

In [ ]:
cmap = "gist_heat"

In [ ]:
figure(figsize=(6, 2))
subplot(121)
imshow(real(a_1_dag_a_1), cmap=cmap)

subplot(122)
imshow(real(unskewed_a_1_a_1_dag), cmap=cmap)
colorbar()

tight_layout()

In [ ]:
Y, X = np.meshgrid(sol.t, sol.t);
rho_11 = real(unskewed_a_1_a_1_dag - unskewed_a_1_dag_a_1)[:, 1:n+1]
# vmin = -1.0 
# vmax =  1.0
vmin = minimum(rho_11)
vmax = maximum(rho_11)

figure(figsize=(5, 3))
ax = plt.gca()
heatmap = ax.pcolormesh(X, Y, (rho_11 |> transpose) .+ rho_11, cmap=cmap, vmin=vmin, vmax=vmax)
ax.set_aspect("equal")
colorbar(mappable=heatmap)
ax.set_xlabel("\$t\$")
ax.set_ylabel("\$t'\$")
ax.set_xlim(0, T)
ax.set_ylim(0, T)
ax.set_xticks([0, T])
ax.set_yticks([0, T])
# ax.legend()
tight_layout()

## Plotting

In [ ]:
t_scale = lambda
ω_scale = 1/lambda
steps = 1
cmap = "gist_heat";

In [ ]:
Y, X = np.meshgrid(sol.t[1:steps:end], sol.t[1:steps:end]);
ρ_11 = imag.(GL.data[1, 1, 1:steps:end, 1:steps:end] - GG.data[1, 1, 1:steps:end, 1:steps:end]);

In [ ]:
length(sol.t)

In [ ]:
xpad = 8
ypad = 5

figure(figsize=(7, 3))

ax = subplot(121)
plot(t_scale * sol.t, [-imag(GL.data[1, 1, k, k]) for k in 1:length(sol.t)], ls="--", c="C3", label="\$i="*string(1)*"\$", lw=1.5)
# plot(t_scale * times, [-imag(GL_T[1, 1][k, k]) for k in 1:n+1], ls=":", c="C1")
# plot(t_scale * times, me.expect[1], c="C0", ls="--", lw=3.0, alpha=0.5)

plot(t_scale * sol.t, [-imag(GL.data[2, 2, k, k]) for k in 1:length(sol.t)], ls="-", c="C0", label="\$i="*string(2)*"\$", lw=1.5)
# plot(t_scale * times, [-imag(GL_T[2, 2][k, k]) for k in 1:n+1], ls=":", c="C1")
# plot(t_scale * times, me.expect[2], c="C2", ls="--", lw=3.0, alpha=0.5)

# plot(times, [real(unskewed_a_1_dag_a_1[k, k]) for k in 1:n+1], c="C0", ls="--", lw=3.0, alpha=0.5)
xlim(0, t_scale * T/2)
xticks(t_scale .* [0, T/4, T/2])
ylim(0, N_0)
xlabel("\$\\lambda t\$")
ylabel("\$-\\mathrm{Im}\\; G^<_{ii}(t, t)\$")
ax.xaxis.set_tick_params(pad=xpad)
ax.yaxis.set_tick_params(pad=ypad)
# ax.set_axisbelow(false)
ax.legend(loc="best", handlelength=1.9, frameon=false, borderpad=0, labelspacing=0.25)
# ticklabel_format(axis="y", style="sci", scilimits=(-0, 0))

ax = subplot(122)
vmin = -0.5 # minimum(ρ_11)
vmax = 1.0 # maximum(ρ_11)
# ax = plt.gca()
heatmap = ax.pcolormesh(t_scale * X, t_scale * Y, ρ_11, cmap=cmap, vmin=vmin, vmax=vmax, rasterized=true)
heatmap.set_edgecolor("face")
ax.set_aspect("equal")
colorbar(mappable=heatmap)
ax.set_xlabel("\$\\lambda t\$")
ax.set_ylabel("\$\\lambda t'\$")
ax.set_xlim(0, t_scale * T)
ax.set_ylim(0, t_scale * T)
ax.set_xticks(t_scale .* [0, T/2, T])
ax.set_yticks(t_scale .* [0, T/2, T])

tight_layout()
savefig("boson_example_T.pdf")#, dpi=256)

### Fourier transform

In [ ]:
using Interpolations
t_lin = range(first(sol.t), stop=last(sol.t), length=length(sol.t))
ρ_11_itp = interpolate((sol.t, sol.t), view(GL.data .- GG.data, 1, 1, :, : ), Gridded(Linear()));
ρ_22_itp = interpolate((sol.t, sol.t), view(GL.data .- GG.data, 2, 2, :, : ), Gridded(Linear()));

In [ ]:
ρ_11_wigner, (taus, ts) = wigner_transform([ρ_11_itp(t1, t2) for t1 in t_lin, t2 in t_lin]; ts=t_lin, fourier=false);
ρ_11_wigner = ρ_11_wigner |> imag;

ρ_22_wigner, (taus, ts) = wigner_transform([ρ_22_itp(t1, t2) for t1 in t_lin, t2 in t_lin]; ts=t_lin, fourier=false);
ρ_22_wigner = ρ_22_wigner |> imag;

In [ ]:
ρ_11_FFT, (ωs, ts) = wigner_transform([ρ_11_itp(t1, t2) for t1 in t_lin, t2 in t_lin]; ts=t_lin, fourier=true);
ρ_11_FFT = -ρ_11_FFT |> real;

ρ_22_FFT, (ωs, ts) = wigner_transform([ρ_22_itp(t1, t2) for t1 in t_lin, t2 in t_lin]; ts=t_lin, fourier=true);
ρ_22_FFT = -ρ_22_FFT |> real;

In [ ]:
xpad = 8
ypad = 5

figure(figsize=(7, 3))

ax = subplot(121)
plot(t_scale * taus, ρ_22_wigner[:, Int(floor(n/2))], ls="-", c="C0", label="\$i=2\$", lw=1.5)
plot(t_scale * taus, ρ_11_wigner[:, Int(floor(n/2))], ls="--", c="C3", label="\$i=1\$", lw=1.5) # fixed T
ax.set_xlabel("\$\\lambda \\tau\$")
ax.set_xlim(-t_scale * T/2, t_scale * T/2)
ax.set_ylim(-0.5, 1.0)
ax.set_xticks(t_scale .* [-T/2, -T/4, 0, T/4, T/2])
ax.xaxis.set_tick_params(pad=xpad)
ax.yaxis.set_tick_params(pad=ypad)
ax.set_ylabel("\$A_{ii}(\\tau, T)\$")

ax = subplot(122)
plot(ω_scale * ωs, ρ_11_FFT[:, Int(floor(n/2))], ls="--", c="C3", label="\$i=1\$", lw=1.5)
plot(ω_scale * ωs, ρ_22_FFT[:, Int(floor(n/2))], ls="-", c="C0", label="\$i=2\$", lw=1.5)
ax.set_xlabel("\$\\omega/\\lambda\$")
ax.set_xlim(10 .* (-1, 1))
ax.set_ylim(0, 2.0)
# ax.set_xticks([-16, -8, 0, 8, 16])
ax.set_xticks([-10, -5, 0, 5, 10])
ax.xaxis.set_tick_params(pad=xpad)
ax.yaxis.set_tick_params(pad=ypad)
ax.set_ylabel("\$A_{ii}(\\omega, T)\$", labelpad=16)
ax.yaxis.set_label_position("right")
ax.legend(loc="best", handlelength=1.4, frameon=false, borderpad=0, labelspacing=0.25)

tight_layout(pad=0.1, w_pad=0.5, h_pad=0)

savefig("boson_example_tau_omega.pdf")

In [ ]:
X, Y = np.meshgrid(ts[1:steps:end], taus[1:steps:end]);

vmin = 1.0 # minimum(ρ_11_wigner)
vmax = -0.5 # maximum(ρ_11_wigner)

figure(figsize=(4, 3))
ax = plt.gca()
heatmap = ax.pcolormesh(t_scale * X, t_scale * Y, ρ_11_wigner[1:steps:end, 1:steps:end], cmap=cmap, vmin=vmin, vmax=vmax, rasterized=true)
heatmap.set_edgecolor("face")
ax.set_xlabel("\$\\lambda T\$")
ax.set_ylabel("\$\\lambda \\tau\$")
ax.set_xlim(0, t_scale * T)
ax.set_xticks([0, t_scale * T/2, t_scale * T])
ax.set_ylim(-t_scale * T/2, t_scale * T/2)
ax.set_yticks([-t_scale * T/2, 0, t_scale * T/2])
colorbar(mappable=heatmap)
ax.set_aspect("equal")
tight_layout(pad=0.0, w_pad=0, h_pad=0)
savefig("boson_example_T_tau.pdf")

In [ ]:
X, Y = np.meshgrid(ts, ωs);

vmin = minimum(ρ_11_FFT)
vmax = maximum(ρ_11_FFT)

figure(figsize=(5, 3))
ax = plt.gca()
heatmap = ax.pcolormesh(t_scale * X, Y, ρ_11_FFT, cmap=cmap, vmin=vmin, vmax=vmax, rasterized=true)
heatmap.set_edgecolor("face")
ax.set_xlabel("\$\\lambda T\$")
ax.set_ylabel("\$\\omega\$")
ax.set_xlim(0, t_scale * T)
ax.set_xticks([0, t_scale * T/2, t_scale * T])
ax.set_yticks([-20, -10, 0, 10, 20])
ax.set_ylim(-20, 20)
colorbar(mappable=heatmap)
tight_layout(pad=0.0, w_pad=0, h_pad=0)

## Testing

In [ ]:
xpad = 8
ypad = 5

figure(figsize=(7, 3))

ax = subplot(121)
plot(t_scale * sol.t, [-imag(GL.data[1, 1, k, k]) for k in 1:length(sol.t)], ls="-", c="C0", label="\$i="*string(1)*"\$")
# plot(t_scale * times, [-imag(GL_T[1, 1][k, k]) for k in 1:n+1], ls=":", c="C1")
plot(t_scale * times, me.expect[1], c="C0", ls="--", lw=3.0, alpha=0.5)

plot(t_scale * sol.t, [-imag(GL.data[2, 2, k, k]) for k in 1:length(sol.t)], ls="-", c="C2", label="\$i="*string(2)*"\$")
# plot(t_scale * times, [-imag(GL_T[2, 2][k, k]) for k in 1:n+1], ls=":", c="C1")
plot(t_scale * times, me.expect[2], c="C2", ls="--", lw=3.0, alpha=0.5)

# plot(times, [real(unskewed_a_1_dag_a_1[k, k]) for k in 1:n+1], c="C0", ls="--", lw=3.0, alpha=0.5)
xlim(0, t_scale * T)
ylim(0, N_0)
xlabel("\$\\lambda t\$")
ylabel("\$-\\mathrm{Im}\\; G^<_{ii}(t, t)\$")
ax.xaxis.set_tick_params(pad=xpad)
ax.yaxis.set_tick_params(pad=ypad)
ax.set_axisbelow(false)
ax.legend(frameon=false)
ticklabel_format(axis="y", style="sci", scilimits=(-0, 0))

ax = subplot(122)
idx = 1
plot(lambda * sol.t, [-imag(GL.data[1, 1, k, k]) for k in 1:length(sol.t)] 
    .- me.expect[1], marker="", ms=3.0, ls="-", c="r")
# xlim(0, 1)
# ylim(-1e-5, 1e-5)
# xticks(J .* [0, 1, 2, 3, 4, 5])
xlabel("\$\\lambda t\$")
ylabel("\$\\mathrm{Im}\\left[ G^<_{11}(t, t) - \\mathcal{G}^<_{11}(t, t) \\right]\$", labelpad=16)
ax.xaxis.set_tick_params(pad=xpad)
ax.yaxis.set_tick_params(pad=ypad)
ax.yaxis.set_label_position("right")
ax.set_axisbelow(false)
# ax.yaxis.tick_right()
ticklabel_format(axis="y", style="sci", scilimits=(-0, 0))

tight_layout(pad=0.2, w_pad=0, h_pad=0)

In [ ]:
figure(figsize=(7, 3))

Y, X = np.meshgrid(sol.t, sol.t);

vmin = -0.5 # minimum(ρ_11)
vmax = 1.0 # maximum(ρ_11)
# ax = plt.gca()
heatmap = imshow(ρ_11, cmap=cmap, vmin=vmin, vmax=vmax, 
    extent=[minimum(X), maximum(X), minimum(Y), maximum(Y)], interpolation="nearest")
# ax.set_aspect("equal")
colorbar(mappable=heatmap)
ax.set_xlabel("\$\\lambda t\$")
ax.set_ylabel("\$\\lambda t'\$")
ax.set_xlim(0, t_scale * T)
ax.set_ylim(0, t_scale * T)
ax.set_xticks([0, t_scale * T])
ax.set_yticks([0, t_scale * T])

tight_layout(pad=0.25, w_pad=0, h_pad=0)

In [ ]:
xpad = 8
ypad = 5

figure(figsize=(7, 3))

ax = subplot(121)
plot(t_scale * sol.t, [-imag(GL.data[1, 1, k, k]) for k in 1:length(sol.t)], ls="--", c="C3", label="\$i="*string(1)*"\$", lw=1.5)
# plot(t_scale * times, [-imag(GL_T[1, 1][k, k]) for k in 1:n+1], ls=":", c="C1")
# plot(t_scale * times, me.expect[1], c="C0", ls="--", lw=3.0, alpha=0.5)

plot(t_scale * sol.t, [-imag(GL.data[2, 2, k, k]) for k in 1:length(sol.t)], ls="-", c="C0", label="\$i="*string(2)*"\$", lw=1.5)
# plot(t_scale * times, [-imag(GL_T[2, 2][k, k]) for k in 1:n+1], ls=":", c="C1")
# plot(t_scale * times, me.expect[2], c="C2", ls="--", lw=3.0, alpha=0.5)

# plot(times, [real(unskewed_a_1_dag_a_1[k, k]) for k in 1:n+1], c="C0", ls="--", lw=3.0, alpha=0.5)
xlim(0, t_scale * T)
ylim(0, N_0)
xlabel("\$\\lambda t\$")
ylabel("\$-\\mathrm{Im}\\; G^<_{ii}(t, t)\$")
ax.xaxis.set_tick_params(pad=xpad)
ax.yaxis.set_tick_params(pad=ypad)
# ax.set_axisbelow(false)
ax.legend(loc="best", handlelength=1.9, frameon=false, borderpad=0, labelspacing=0.25)
# ticklabel_format(axis="y", style="sci", scilimits=(-0, 0))

ax = subplot(122)
idx = 1
plot(t_scale * sol.t, [imag(GL.data[1, 2, k, k]) for k in 1:length(sol.t)], ls="-", c="k", label="\$\\mathrm{Re}\$", lw=1.5)
plot(t_scale * sol.t, [real(GL.data[1, 2, k, k]) for k in 1:length(sol.t)], ls="-.", c="k", label="\$\\mathrm{Im}\$", lw=1.5)
# plot(t_scale * times, [-imag(GL_T[1, 1][k, k]) for k in 1:n+1] 
#     .- me.expect[1], marker="", ms=3.0, ls=":", c="r")
xlim(0, t_scale * T)
# ylim(-0.1, 0.1)
xlabel("\$\\lambda t\$")
ylabel("\$G^>_{12}(t, t)\$", labelpad=16)
ax.xaxis.set_tick_params(pad=xpad)
ax.yaxis.set_tick_params(pad=ypad)
ax.yaxis.set_label_position("right")
# ax.set_axisbelow(false)
# ax.yaxis.tick_right()
ax.legend(loc="best", handlelength=1.9, frameon=false, borderpad=0, labelspacing=0.25)
# ticklabel_format(axis="y", style="sci", scilimits=(-0, 0))

tight_layout(pad=0.2, w_pad=0.5, h_pad=0)